In [23]:
import cv2
from utils import display, resizeWithAspectRatio, imageOverlay
import numpy as np
import time

In [34]:
img = cv2.imread('scaled/field.png')
img = cv2.imread("scaled/IMG_1133.jpg")
#img = cv2.imread("scaled/scaleImage2.jpg")
#img = cv2.imread('scaled/field2.png')
img = cv2.imread('scaled/scaleImage7.jpg')

start = time.time()

# resize img for speed
height, width = img.shape[:2]
img = cv2.resize(img, (round(width / 5), round(height / 5)), interpolation=cv2.INTER_AREA)
imgBlurred = cv2.GaussianBlur(img, (5, 5), 0)

imgGray2 = cv2.cvtColor(imgBlurred.copy(), cv2.COLOR_BGR2GRAY)
_, thresh2 = cv2.threshold(imgGray2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

Z = imgBlurred.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4
ret,label,center=cv2.kmeans(Z,K,None,criteria,1,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

# convert to black and white
imgGray = cv2.cvtColor(res2, cv2.COLOR_BGR2GRAY)
thresh, imgf = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

thresh, imgf = cv2.threshold(imgGray, 180, 255, cv2.THRESH_BINARY)

print(time.time() - start)

display(cv2.threshold(imgGray2, 180, 255, cv2.THRESH_BINARY)[1])
display(thresh2)
display(imgBlurred)
display(res2)
display(imgf)

cv2.waitKey(0)
cv2.destroyAllWindows()

0.06199932098388672


In [25]:
# plot a histogram of an image
for i in cv2.calcHist([wimg],[0],None,[256],[0,256]):
    print(f"{i[0]},")

NameError: name 'wimg' is not defined

In [65]:
type(imgf)

numpy.ndarray

In [77]:
contours, hierarchy = cv2.findContours(imgf, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
cv2.drawContours(imgf, contours, -1, (0,255,0), 10)
display(imgf)

In [80]:
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

In [83]:
wimg= white_balance(img)

In [17]:
def imageOverlay(background, overlay ,width=0, x=0, y=0):
    """ overlay an image over another """

    dim = (width, width)
    # resize image
    overlay = cv2.resize(overlay, dim, interpolation = cv2.INTER_AREA)
    
    rows,cols,channels = overlay.shape
    overlay=cv2.addWeighted(background[x:x+rows, y:y+cols],0,overlay,1,0)
    background[250:250+rows, 0:0+cols ] = overlay

    return background

In [24]:
overlay = cv2.imread('test_images/target.jpg')
background = cv2.imread('test_images/field2.png')
display(overlay_transparent(background, overlay, x=100, y=100, width=50))

In [7]:
target.shape

(388, 394, 3)

In [8]:
field.shape

(912, 1283, 3)

In [22]:
import cv2
import numpy as np

def overlay_transparent(background, overlay, x, y, width):
    
    dim = (width, width)
    # resize image
    overlay = cv2.resize(overlay, dim, interpolation = cv2.INTER_AREA)

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [25]:
a = np.array([1])

In [26]:
a.tolist()


[1]

In [19]:
# add barrel distortion

import numpy as np
import cv2

src    = cv2.imread("test_images/field.png")

width  = src.shape[1]
height = src.shape[0]

distCoeff = np.zeros((4,1),np.float64)

# TODO: add your coefficients here!
k1 = -2.7e-5; # negative to remove barrel distortion
k2 = 0.0;
p1 = 0.0;
p2 = 0.0;

distCoeff[0,0] = k1;
distCoeff[1,0] = k2;
distCoeff[2,0] = p1;
distCoeff[3,0] = p2;

# assume unit matrix for camera
cam = np.eye(3,dtype=np.float32)

cam[0,2] = width/2.0  # define center x
cam[1,2] = height/2.0 # define center y
cam[0,0] = 10.        # define focal length x
cam[1,1] = 10.        # define focal length y

# here the undistortion will be computed
dst = cv2.undistort(src,cam,distCoeff)

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [7]:
import target_recognition
k_nearest = target_recognition.load_model()
target_recognition.k_nearest = k_nearest
target_recognition.find_characters(dst)

0.00099945068359375 to process letter T


[['T', [14.121631622314453, 4.800229072570801, 246.02029418945312]]]

In [9]:
def distort(img, coef):
    
    width  = src.shape[1]
    height = src.shape[0]

    distCoeff = np.zeros((4,1),np.float64)

    # TODO: add your coefficients here!
    k1 = coef; # negative to remove barrel distortion
    k2 = 0.0;
    p1 = 0.0;
    p2 = 0.0;

    distCoeff[0,0] = k1;
    distCoeff[1,0] = k2;
    distCoeff[2,0] = p1;
    distCoeff[3,0] = p2;

    # assume unit matrix for camera
    cam = np.eye(3,dtype=np.float32)

    cam[0,2] = width/2.0  # define center x
    cam[1,2] = height/2.0 # define center y
    cam[0,0] = 10.        # define focal length x
    cam[1,1] = 10.        # define focal length y

    # here the undistortion will be computed
    dst = cv2.undistort(src,cam,distCoeff)
    
    return dst

In [13]:
src = cv2.imread("test_images/field.png")
coef = -1e-5
results = []
for i in range(100):
    coef -= 0.1e-5
    img = distort(src, coef)
    r = target_recognition.find_characters(img)
    results.append([coef, r])

0.0009996891021728516 to process letter T
0.001998424530029297 to process letter T
0.0019989013671875 to process letter T
0.0019991397857666016 to process letter T
0.0009996891021728516 to process letter T
0.00099945068359375 to process letter T
0.0009992122650146484 to process letter T
0.0009996891021728516 to process letter T
0.0009996891021728516 to process letter T
0.0019991397857666016 to process letter T
0.0009996891021728516 to process letter T
0.00099945068359375 to process letter T
0.0019989013671875 to process letter T
0.0019989013671875 to process letter T
0.0009996891021728516 to process letter T
0.00099945068359375 to process letter T
0.001998424530029297 to process letter T


In [14]:
results 

[[-1.1000000000000001e-05,
  [['T', [13.777358055114746, 4.3942179679870605, 246.81289672851562]]]],
 [-1.2000000000000002e-05,
  [['T', [13.594133377075195, 4.2114667892456055, 246.723876953125]]]],
 [-1.3000000000000003e-05,
  [['T', [13.583355903625488, 4.211262226104736, 246.3133087158203]]]],
 [-1.4000000000000003e-05,
  [['T', [13.676854133605957, 4.286823749542236, 246.97320556640625]]]],
 [-1.5000000000000004e-05,
  [['T', [13.579848289489746, 4.197278022766113, 246.70384216308594]]]],
 [-1.6000000000000003e-05,
  [['T', [13.949897766113281, 4.603034973144531, 246.2431640625]]]],
 [-1.7000000000000003e-05,
  [['T', [13.098544120788574, 3.6736068725585938, 247.27197265625]]]],
 [-1.8000000000000004e-05,
  [['T', [13.387857437133789, 3.981649398803711, 247.07861328125]]]],
 [-1.9000000000000004e-05,
  [['T', [13.830865859985352, 4.4659743309021, 246.4813232421875]]]],
 [-2.0000000000000005e-05,
  [['T', [14.121631622314453, 4.800229072570801, 246.02029418945312]]]],
 [-2.10000000

In [34]:
def noisy(noise_typ,image):
    """
    Parameters
    ----------
    image : ndarray
        Input image data. Will be converted to float.
    mode : str
        One of the following strings, selecting the type of noise to add:

        'gauss'     Gaussian-distributed additive noise.
        'poisson'   Poisson-distributed noise generated from the data.
        's&p'       Replaces random pixels with 0 or 1.
        'speckle'   Multiplicative noise using out = image + n*image,where
                    n is uniform noise with specified mean & variance.
    """
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

def sp_noise(image, prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = image.copy()
    if len(image.shape) == 2:
        black = 0
        white = 255            
    else:
        colorspace = image.shape[2]
        if colorspace == 3:  # RGB
            black = np.array([0, 0, 0], dtype='uint8')
            white = np.array([255, 255, 255], dtype='uint8')
        else:  # RGBA
            black = np.array([0, 0, 0, 255], dtype='uint8')
            white = np.array([255, 255, 255, 255], dtype='uint8')
    probs = np.random.random(image.shape[:2])
    image[probs < (prob / 2)] = black
    image[probs > 1 - (prob / 2)] = white
    return image

In [40]:
import random
src = cv2.imread("test_images/field.png")
img = sp_noise(src, 0.05)
display(img)
coef = 0
results = []
for i in range(100):
    coef += 0.001
    img = sp_noise(src, coef)
    r = target_recognition.find_characters(img)
    results.append([coef, r])

0.0010023117065429688 to process letter T
0.0010008811950683594 to process letter T
0.0019998550415039062 to process letter T
0.0019989013671875 to process letter T
0.0009996891021728516 to process letter T
0.0020079612731933594 to process letter T
0.0019996166229248047 to process letter T
0.0010001659393310547 to process letter T
0.002000093460083008 to process letter T
0.000993490219116211 to process letter T
0.0010001659393310547 to process letter T


In [42]:
import random
src = cv2.imread("test_images/field.png")
img = sp_noise(src, 0.011)
display(img)


In [48]:
src = cv2.imread("test_images/field.png")

coef = 1
results = []
for i in range(100):
    coef += 2
    img = cv2.GaussianBlur(src,(coef,coef),0)
    r = target_recognition.find_characters(img)
    results.append([coef, r])

0.0009999275207519531 to process letter T
0.0020003318786621094 to process letter T
0.0010075569152832031 to process letter T
0.0010013580322265625 to process letter T
0.0010008811950683594 to process letter T
0.0020003318786621094 to process letter T
0.0020008087158203125 to process letter T
0.001999378204345703 to process letter T
0.0009996891021728516 to process letter T
0.0009996891021728516 to process letter T


In [50]:
img = cv2.GaussianBlur(src,(21,21),0)
display(img)